In [1]:
# Huggingface dataset
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

In [3]:
# Access LIAR dataset via Huggingface, split dataset into train/test/validation with ratio 8:1:1
dataset = load_dataset('liar')
train_dataset = dataset['train']
test_dataset = dataset['test']
validation_dataset = dataset['validation']

In [5]:
print("train__________________________________________")
for index, data in enumerate(train_dataset):
    if index == 5:
        break
    print(data)
print("test__________________________________________")
for index, data in enumerate(test_dataset):
    if index == 5:
        break
    print(data)
print("validation__________________________________________")
for index, data in enumerate(validation_dataset):
    if index == 5:
        break
    print(data)

train__________________________________________
{'id': '2635.json', 'label': 0, 'statement': 'Says the Annies List political group supports third-trimester abortions on demand.', 'subject': 'abortion', 'speaker': 'dwayne-bohac', 'job_title': 'State representative', 'state_info': 'Texas', 'party_affiliation': 'republican', 'barely_true_counts': 0.0, 'false_counts': 1.0, 'half_true_counts': 0.0, 'mostly_true_counts': 0.0, 'pants_on_fire_counts': 0.0, 'context': 'a mailer'}
{'id': '10540.json', 'label': 1, 'statement': 'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.', 'subject': 'energy,history,job-accomplishments', 'speaker': 'scott-surovell', 'job_title': 'State delegate', 'state_info': 'Virginia', 'party_affiliation': 'democrat', 'barely_true_counts': 0.0, 'false_counts': 0.0, 'half_true_counts': 1.0, 'mostly_true_counts': 1.0, 'pants_on_fire_counts': 0.0, 'context': 'a floor speech.'}
{'id':

In [6]:
import pandas as pd

In [7]:
# Extract data to use, frame into pandas dataframe for training/test
data = {"label": train_dataset['label'], "statement": train_dataset['statement'], "subject": train_dataset['subject']}
pd_train_dataset = pd.DataFrame(data)
data = {"label": test_dataset['label'], "statement": test_dataset['statement'], "subject": test_dataset['subject']}
pd_test_dataset = pd.DataFrame(data)
data = {"label": validation_dataset['label'], "statement": validation_dataset['statement'], "subject": validation_dataset['subject']}
pd_validation_dataset = pd.DataFrame(data)

print(pd_train_dataset.iloc[0])
print(pd_test_dataset.iloc[0])
print(pd_validation_dataset.iloc[0])

label                                                        0
statement    Says the Annies List political group supports ...
subject                                               abortion
Name: 0, dtype: object
label                                                        3
statement    Building a wall on the U.S.-Mexico border will...
subject                                            immigration
Name: 0, dtype: object
label                                                        4
statement    We have less Americans working now than in the...
subject                                           economy,jobs
Name: 0, dtype: object


In [8]:
import subprocess

In [ ]:
# To process LIAR dataset with LIWC software for text analysis
with open('train_analysis.txt', 'a') as f:
    length = len(pd_validation_dataset["statement"])
    for index in range(length):
        sentence = pd_validation_dataset["statement"][index]
        cmd_to_execute = [
            "LIWC-22-cli",
            "--mode", "wc",
            "--input", "console",
            "--console-text", sentence,
            "--output", "console"
        ]
        try:
            results = subprocess.check_output(cmd_to_execute, stderr=subprocess.DEVNULL).decode().strip().splitlines()
            print(f"{index}/{length}")
            f.write(results[7])
            f.write('\n\n')
        except Exception as e:
            print(f"Error processing sentence: {sentence}. Error: {e}")

with open('train_analysis.txt', 'a') as f:
    length = len(pd_validation_dataset["statement"])
    for index in range(length):
        sentence = pd_validation_dataset["statement"][index]
        cmd_to_execute = [
            "LIWC-22-cli",
            "--mode", "wc",
            "--input", "console",
            "--console-text", sentence,
            "--output", "console"
        ]
        try:
            results = subprocess.check_output(cmd_to_execute, stderr=subprocess.DEVNULL).decode().strip().splitlines()
            print(f"{index}/{length}")
            f.write(results[7])
            f.write('\n\n')
        except Exception as e:
            print(f"Error processing sentence: {sentence}. Error: {e}")

with open('train_analysis.txt', 'a') as f:
    length = len(pd_validation_dataset["statement"])
    for index in range(length):
        sentence = pd_validation_dataset["statement"][index]
        cmd_to_execute = [
            "LIWC-22-cli",
            "--mode", "wc",
            "--input", "console",
            "--console-text", sentence,
            "--output", "console"
        ]
        try:
            results = subprocess.check_output(cmd_to_execute, stderr=subprocess.DEVNULL).decode().strip().splitlines()
            print(f"{index}/{length}")
            f.write(results[7])
            f.write('\n\n')
        except Exception as e:
            print(f"Error processing sentence: {sentence}. Error: {e}")